<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 02 - Preprocessing
</div>

In [1]:
import pandas as pd
import numpy as np
import regex as re
import ast 

1. Đọc dữ liệu từ file đã draw và xem xét những thông tin chung của dữ liệu

In [2]:
df = pd.read_csv('../data/raw_data.csv')
df.shape

(984, 16)

In [3]:
df.columns

Index(['job_title', 'company_name', 'salary', 'address', 'date_posted',
       'industry', 'company_size', 'company_nationality', 'experience_years',
       'position_level', 'employment_type', 'contract_type',
       'technologies_used', 'job_description', 'company_url', 'job_url'],
      dtype='object')

In [4]:
df.dtypes

job_title              object
company_name           object
salary                 object
address                object
date_posted            object
industry               object
company_size           object
company_nationality    object
experience_years       object
position_level         object
employment_type        object
contract_type          object
technologies_used      object
job_description        object
company_url            object
job_url                object
dtype: object

2. Xóa các thuộc tính không cần thiết cho việc xây dụng mô hình

In [5]:
df = df.drop(columns=['date_posted'])

In [6]:
df.head()

,job_title,company_name,salary,address,industry,company_size,company_nationality,experience_years,position_level,employment_type,contract_type,technologies_used,job_description,company_url,job_url
0,UX/UI Designer (Game),CÔNG TY CỔ PHẦN SUNTEK,Thương lượng,"['Thành phố Thủ Đức, Hồ Chí Minh']",['Giải trí/ Game'],['25-99 Nhân viên'],['Thailand'],['Từ 2 năm'],"['Junior', 'Middle']",['In Office'],['Fulltime'],"['UX/UI Design', 'HTML & CSS', 'UI/UX']",['Trách nhiệm công việc\n1. General task\nHave...,['https://topdev.vn/nha-tuyen-dung/cong-ty-co-...,https://topdev.vn/viec-lam/ux-ui-designer-game...
1,Senior DevOps Engineer,CÔNG TY CỔ PHẦN CÔNG NGHỆ ỨNG DỤNG XANH VIỆT NAM,40.000.000 VND to 50.000.000 VND,"['Quận Ba Đình, Hà Nội']",[],['Nhân viên'],[],['Từ 5 năm'],['Senior'],['In Office'],['Fulltime'],"['DevOps', 'Docker', 'Ansible', 'CI/CD', 'Terr...","['GA là một công ty khởi nghiệp, phát triển và...",['https://topdev.vn/nha-tuyen-dung/cong-ty-co-...,https://topdev.vn/viec-lam/senior-devops-engin...
2,KỸ SƯ PHẦN MỀM (C# Developer) - Urgent,Aram Vina,Lên tới 1.500 VND,"['Quận Tây Hồ, Hà Nội']","['Thiết Kế, Software, Gia công phần mềm, Triển...",['25-99 Nhân viên'],['South-Korea'],['Không yêu cầu'],"['Fresher', 'Junior', 'Middle']",['In Office'],['Fulltime'],"['C#', 'WPF', 'Software Developer']",['CÔNG TY TNHH ARAM VINA cần tuyển dụng GẤP KỸ...,['https://topdev.vn/nha-tuyen-dung/aram-vina-9...,https://topdev.vn/viec-lam/ky-su-phan-mem-c-de...
3,Chuyên viên Thẩm định mua sắm CNTT,Ban Công nghệ BIDV,Thương lượng,"['Quận Hai Bà Trưng, Hà Nội']",['Ngân Hàng'],['Hơn 1000 Nhân viên'],['Vietnam'],['Từ 3 năm'],['Middle'],['In Office'],['Fulltime'],"['IT Support', 'IT Service']",['Trách nhiệm công việc\nThực hiện công tác th...,['https://topdev.vn/nha-tuyen-dung/ban-cong-ng...,https://topdev.vn/viec-lam/chuyen-vien-tham-di...
4,Chuyên viên Quản lý Chuyển đổi số,Ban Công nghệ BIDV,Thương lượng,"['Quận Hai Bà Trưng, Hà Nội']",['Ngân Hàng'],['Hơn 1000 Nhân viên'],['Vietnam'],['Từ 2 năm'],"['Junior', 'Middle']",['In Office'],['Fulltime'],"['Agile', 'Project Manager', 'Product Owner', ...",['Trách nhiệm công việc\nPhối hợp triển khai t...,['https://topdev.vn/nha-tuyen-dung/ban-cong-ng...,https://topdev.vn/viec-lam/chuyen-vien-quan-ly...


3. Xử lý giá trị null

In [7]:
df.isnull().sum()

job_title              0
company_name           0
salary                 1
address                0
industry               0
company_size           0
company_nationality    0
experience_years       0
position_level         0
employment_type        0
contract_type          0
technologies_used      0
job_description        0
company_url            0
job_url                0
dtype: int64

- Vì giá trị null quá ít nên ta sẽ xóa những dòng có giá trị null

In [8]:
df = df.dropna()

4. Xem xét các giá trị tiêu biểu của mỗi thuộc tính

In [9]:
def format_percentages(col):
    percentages = col.value_counts(normalize=True).mul(100).round(2)
    return ', '.join([f"{idx}: {val}%" for idx, val in percentages.items()])

formatted = df.apply(format_percentages)
print(formatted)


job_title              Java Developer: 1.22%, Business Analyst: 1.02%...
company_name           MBBANK: 12.92%, LG CNS Việt Nam: 4.48%, Tổng C...
salary                 Thương lượng: 69.79%, 20.000.000 VND to 30.000...
address                ['Quận Cầu Giấy, Hà Nội']: 18.62%, ['Quận Nam ...
industry               ['Ngân Hàng']: 22.08%, ['Phần Mềm']: 15.36%, [...
company_size           ['Hơn 1000 Nhân viên']: 35.91%, ['25-99 Nhân v...
company_nationality    ['Vietnam']: 62.67%, ['South-Korea']: 7.43%, [...
experience_years       ['Từ 2 năm']: 22.89%, ['Từ 1 năm']: 22.89%, ['...
position_level         ['Junior', 'Middle']: 21.57%, ['Middle', 'Seni...
employment_type        ['In Office']: 93.39%, ['Hybrid']: 3.36%, ['Re...
contract_type          ['Fulltime']: 99.39%, ['Freelance']: 0.31%, ['...
technologies_used      ['Business Analyst']: 1.32%, ['IT Support', 'T...
job_description        ['Trách nhiệm công việc\nPhát triển và bảo trì...
company_url            ['https://topdev.vn/nha-tuye

5. Xử lý thuộc tính salary

In [10]:
percentages = df['salary'].value_counts(normalize=True).mul(100)
percentages

salary
Thương lượng                        69.786368
20.000.000 VND to 30.000.000 VND     1.831129
Lên tới 2.500 USD                    1.525941
10.000.000 VND to 20.000.000 VND     1.322482
20.000.000 VND to 40.000.000 VND     1.017294
                                      ...    
12.000.000 VND to 42.000.000 VND     0.305188
20.000.000 VND to 50.000.000 VND     0.305188
17.000.000 VND to 25.000.000 VND     0.305188
Lên tới 1.500 USD                    0.305188
3.000.000 VND to 5.000.000 VND       0.305188
Name: proportion, Length: 63, dtype: float64

- Mức lương "Thương lượng" sẽ được gán là NaN

In [11]:
df['salary'] = df['salary'].replace("Thương lượng", np.nan)

- Các mức lương còn lại sẽ được chuẩn hóa theo kiểu dữ liệu số

In [12]:
def extract_salary(s):
    matches = re.findall(r"(\d{1,3}(?:\.\d{3})+)", s)
    if len(matches) == 1:
        return int(matches[0].replace('.',''))
    elif len(matches) == 2:
        min_salary = int(matches[0].replace('.', ''))
        max_salary = int(matches[1].replace('.', ''))
        return np.array([min_salary, max_salary])
mask = df['salary'].notna()

df.loc[mask, 'salary'] = df.loc[mask, 'salary'].apply(extract_salary)


- Chuyển giá trị USD thành VND

In [13]:
percentages = df['salary'].value_counts(normalize=True).mul(100)
percentages.head()

salary
2500        5.102041
35000000    3.401361
1500        3.401361
18000000    3.061224
50000000    2.380952
Name: proportion, dtype: float64

In [14]:
mask = df['salary'].apply(
    lambda x: x if isinstance(x, int)
    else x[0] if isinstance(x, list)
    else np.inf
) < 10000
df.loc[mask, 'salary'] = df.loc[mask, 'salary'] * 25000
df['salary'].head()

0                     NaN
1    [40000000, 50000000]
2                37500000
3                     NaN
4                     NaN
Name: salary, dtype: object

6. Xử lý thuộc tính `experience_years`

In [15]:
def extract_experience (s):
    reg = r'\d+'
    numbers = re.findall(reg, s) 
    if len(numbers) == 0:
        return 0
    elif len(numbers) == 1:
        return int(numbers[0])
    else: 
        return np.array([int(numbers[0]), int(numbers[1])])
df['experience_years'] = df['experience_years'].apply(extract_experience)
df['experience_years'].head(20)

0      2
1      5
2      0
3      3
4      2
5      5
6      3
7      2
8      3
9     10
10     3
11     7
12     5
13     3
14     1
15     1
16     0
17     3
18     1
19     2
Name: experience_years, dtype: int64

7. Xử lý thuộc tính `company_size`

In [16]:
def extract_company_size(s):
    reg = r'\d+'
    numbers = re.findall(reg, s) 
    if len(numbers) == 0:
        return np.nan
    elif len(numbers) == 1:
        return int(numbers[0])
    else: 
        return np.array([int(numbers[0]), int(numbers[1])])
df['company_size'] = df['company_size'].apply(extract_company_size)
df['company_size'].head(10)

0    [25, 99]
1         NaN
2    [25, 99]
3        1000
4        1000
5        1000
6        1000
7        1000
8        1000
9        1000
Name: company_size, dtype: object

In [17]:
df.head()

,job_title,company_name,salary,address,industry,company_size,company_nationality,experience_years,position_level,employment_type,contract_type,technologies_used,job_description,company_url,job_url
0,UX/UI Designer (Game),CÔNG TY CỔ PHẦN SUNTEK,NaN,"['Thành phố Thủ Đức, Hồ Chí Minh']",['Giải trí/ Game'],"[25, 99]",['Thailand'],2,"['Junior', 'Middle']",['In Office'],['Fulltime'],"['UX/UI Design', 'HTML & CSS', 'UI/UX']",['Trách nhiệm công việc\n1. General task\nHave...,['https://topdev.vn/nha-tuyen-dung/cong-ty-co-...,https://topdev.vn/viec-lam/ux-ui-designer-game...
1,Senior DevOps Engineer,CÔNG TY CỔ PHẦN CÔNG NGHỆ ỨNG DỤNG XANH VIỆT NAM,"[40000000, 50000000]","['Quận Ba Đình, Hà Nội']",[],NaN,[],5,['Senior'],['In Office'],['Fulltime'],"['DevOps', 'Docker', 'Ansible', 'CI/CD', 'Terr...","['GA là một công ty khởi nghiệp, phát triển và...",['https://topdev.vn/nha-tuyen-dung/cong-ty-co-...,https://topdev.vn/viec-lam/senior-devops-engin...
2,KỸ SƯ PHẦN MỀM (C# Developer) - Urgent,Aram Vina,37500000,"['Quận Tây Hồ, Hà Nội']","['Thiết Kế, Software, Gia công phần mềm, Triển...","[25, 99]",['South-Korea'],0,"['Fresher', 'Junior', 'Middle']",['In Office'],['Fulltime'],"['C#', 'WPF', 'Software Developer']",['CÔNG TY TNHH ARAM VINA cần tuyển dụng GẤP KỸ...,['https://topdev.vn/nha-tuyen-dung/aram-vina-9...,https://topdev.vn/viec-lam/ky-su-phan-mem-c-de...
3,Chuyên viên Thẩm định mua sắm CNTT,Ban Công nghệ BIDV,NaN,"['Quận Hai Bà Trưng, Hà Nội']",['Ngân Hàng'],1000,['Vietnam'],3,['Middle'],['In Office'],['Fulltime'],"['IT Support', 'IT Service']",['Trách nhiệm công việc\nThực hiện công tác th...,['https://topdev.vn/nha-tuyen-dung/ban-cong-ng...,https://topdev.vn/viec-lam/chuyen-vien-tham-di...
4,Chuyên viên Quản lý Chuyển đổi số,Ban Công nghệ BIDV,NaN,"['Quận Hai Bà Trưng, Hà Nội']",['Ngân Hàng'],1000,['Vietnam'],2,"['Junior', 'Middle']",['In Office'],['Fulltime'],"['Agile', 'Project Manager', 'Product Owner', ...",['Trách nhiệm công việc\nPhối hợp triển khai t...,['https://topdev.vn/nha-tuyen-dung/ban-cong-ng...,https://topdev.vn/viec-lam/chuyen-vien-quan-ly...


- Ghi dữ liệu lại

In [18]:
df.to_csv('../data/preprocessed_data.csv')